In [2]:
pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [23]:
# Import necesary libraries
import pandas_datareader.data as pdr
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [24]:
# Download historical data for required stocks
ticker = "HDFCBANK.NS"
ohlcv = pdr.get_data_yahoo(ticker,datetime.date.today()-datetime.timedelta(1825),datetime.date.today())
ohlcv.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-10-13,639.500000,628.224976,639.500000,631.375000,2259678.0,615.338501
2016-10-14,634.000000,627.000000,632.875000,630.900024,2033558.0,614.875549
2016-10-17,633.200012,616.375000,632.750000,618.849976,2567680.0,603.131531
2016-10-18,630.000000,619.500000,621.599976,628.924988,1305484.0,612.950684
2016-10-19,631.500000,624.000000,630.549988,629.174988,1522452.0,613.194397


In [25]:
# Download historical data for required stocks
ticker = "RELIANCE.NS"
ohlcv = pdr.get_data_yahoo(ticker,datetime.date.today()-datetime.timedelta(1825),datetime.date.today())
ohlcv.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-10-13,539.411926,521.060852,539.387146,523.611694,10124402.0,510.205902
2016-10-14,535.920044,525.840576,526.509216,533.765442,7045099.0,520.099731
2016-10-17,535.325623,524.404175,534.929382,525.345276,4535058.0,511.895081
2016-10-18,536.291504,526.038696,527.995117,535.474243,5110251.0,521.764709
2016-10-19,542.210388,535.622803,537.133484,538.569885,7879298.0,524.781067


In [26]:
def MACD(DF,a,b,c):
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    df = DF.copy()
    df["MA_Fast"]=df["Adj Close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["Adj Close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df['Day2_Close'] = df['Adj Close'].shift(-1)
    df['Day3_Close'] = df['Adj Close'].shift(-2)
    df['Day4_Close'] = df['Adj Close'].shift(-3)
    df['Day5_Close'] = df['Adj Close'].shift(-4)
    df['Day6_Close'] = df['Adj Close'].shift(-5)
     
    #df['MACD-Signal'] = df["MACD"] - df["Signal"]
    df['Avg_Close'] = (df['Day2_Close'] + df['Day3_Close'] + df['Day4_Close'] + df['Day5_Close'] + df['Day6_Close'])/5
    df['PL'] = df['Avg_Close'] - df['Adj Close']
 

    df.dropna(inplace=True)
    data=df.copy()
    data.reset_index(inplace = True)

    fig = go.Figure(data=[go.Candlestick(x=data['Date'], open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'])])
    fig.show()
         

In [27]:
def RSI(DF,n):
    "function to calculate RSI"
    df = DF.copy()
    df['delta']=df['Adj Close'] - df['Adj Close'].shift(1)
    df['gain']=np.where(df['delta']>=0,df['delta'],0)
    df['loss']=np.where(df['delta']<0,abs(df['delta']),0)
    avg_gain = []
    avg_loss = []
    gain = df['gain'].tolist()
    loss = df['loss'].tolist()
    for i in range(len(df)):
        if i < n:
            avg_gain.append(np.NaN)
            avg_loss.append(np.NaN)
        elif i == n:
            avg_gain.append(df['gain'].rolling(n).mean().tolist()[n])
            avg_loss.append(df['loss'].rolling(n).mean().tolist()[n])
        elif i > n:
            avg_gain.append(((n-1)*avg_gain[i-1] + gain[i])/n)
            avg_loss.append(((n-1)*avg_loss[i-1] + loss[i])/n)
    df['avg_gain']=np.array(avg_gain)
    df['avg_loss']=np.array(avg_loss)
    df['RS'] = df['avg_gain']/df['avg_loss']
    df['RSI'] = 100 - (100/(1+df['RS']))
    
    df.dropna(inplace=True)
    data=df.copy()
    data.reset_index(inplace = True)

    fig = go.Figure(data=[go.Candlestick(x=data['Date'], open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'])])
    fig.show()
    
    return df['RSI']

# Calculating RSI without using loop
def rsi(df, n):
    "function to calculate RSI"
    delta = df["Adj Close"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n] ) #first value is sum of avg gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n] ) #first value is sum of avg losses
    d = d.drop(d.index[:(n-1)])
    rs = pd.stats.moments.ewma(u, com=n-1, adjust=False) / \
         pd.stats.moments.ewma(d, com=n-1, adjust=False)
    return 100 - 100 / (1 + rs)

In [28]:
MACD(ohlcv,12,26,9)

In [29]:
RSI(ohlcv,10)

Date
2016-10-27    45.907939
2016-10-28    49.316841
2016-11-01    47.226822
2016-11-02    35.647143
2016-11-03    35.775566
                ...    
2021-10-05    80.379779
2021-10-06    66.400905
2021-10-07    67.974628
2021-10-08    77.285668
2021-10-11    72.857021
Name: RSI, Length: 1223, dtype: float64